In [1]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import time

import torch
print(torch.cuda.is_available())  # True 表示可以用 GPU

cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"
)
cfg.MODEL.DEVICE = "cuda"  # 指定使用 GPU

predictor = DefaultPredictor(cfg)

print("Detectron2 模型運行裝置:", predictor.model.device)

True
Detectron2 模型運行裝置: cuda:0


In [ ]:
# analyze_pitch_from_detectron2.py
# 👉 全自動流程：影片 → Detectron2 → 三動作偵測 → 特徵分析

import os
import json
import time
import shutil
import pandas as pd
from tqdm import tqdm
import torch
import warnings
warnings.filterwarnings("ignore")

from pose_utils_detectron2 import load_pose_sequence
from pose_detectron2 import run_detectron2

from detect_release import detect_release
from detect_landing import detect_landing
from detect_shoulder import detect_shoulder

from landing_features import detect_landing_features
from shoulder_features import extract_shoulder_features
from release_features import extract_release_features

# ✅ 單支影片轉特徵 json
def video_2_json(video_path, json_path, description, keypoint_dir, device):
    try:
        if os.path.exists(keypoint_dir):
            # 有舊資料，跳過後續動作
            print(f"{keypoint_dir} 已存在，跳過處理。")
            return 0

        # 🧠 執行 detectron2 輸入影片 → 關節座標資料
        run_detectron2(video_path=video_path, output_dir=keypoint_dir, device=device,save_img=False)
        pose_sequence = load_pose_sequence(keypoint_dir)

        # 🎯 偵測三個關鍵偵
        release_frame = detect_release(pose_sequence)
        landing_frame = detect_landing(pose_sequence, release_frame)
        shoulder_frame = detect_shoulder(pose_sequence, release_frame)

        # 🔍 提取所有特徵
        all_features = {}

        landing_features = detect_landing_features(pose_sequence, landing_frame)
        for key, value in landing_features.items():
            all_features[f'landing_features_{key}'] = value

        shoulder_features = extract_shoulder_features(pose_sequence, shoulder_frame, landing_frame)
        for key, value in shoulder_features.items():
            all_features[f'shoulder_features_{key}'] = value

        release_features = extract_release_features(pose_sequence, release_frame)
        for key, value in release_features.items():
            all_features[f'release_features_{key}'] = value

        all_features['description'] = description

        # 💾 輸出 JSON
        assert ('data' in json_path) and ('features_and_labels' in json_path)
        os.makedirs(os.path.dirname(json_path), exist_ok=True)
        with open(json_path, 'w') as f:
            json.dump(all_features, f, indent=2)
        print('✅ 特徵已儲存至', json_path)

    except Exception as e:
        print('❌ 錯誤發生於影片:', video_path)
        print(e)

ROOT_DIR = 'data'

# ✅ 主程式
def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"⚙️ 使用裝置：{device}")

    for subdir in tqdm(os.listdir(ROOT_DIR)):
        subdir_path = os.path.join(ROOT_DIR, subdir)
        if not os.path.isdir(subdir_path):
            print('⚠️ 找不到子目錄')
            continue

        player_name = "_".join(subdir.split("_")[:3])
        csv_path = os.path.join(ROOT_DIR, subdir, f"{player_name}.csv")

        if not os.path.exists(csv_path):
            print(f"⚠️ 找不到對應的 CSV：{csv_path}")
            continue

        df = pd.read_csv(csv_path)

        for video_file in tqdm(os.listdir(subdir_path)):
            if not video_file.endswith('.mp4'):
                continue

            video_path = os.path.join(subdir_path, video_file)

            # ✅ 更穩健的檢查影片對應描述（忽略大小寫）
            row = df[df['Filename'].str.lower() == video_file.lower()]
            if row.empty:
                print(f"⚠️ 找不到影片對應的描述：{video_file}")
                continue

            description = row.iloc[0]['description']

            json_output_path = os.path.join(subdir_path, "features_and_labels", video_file.replace(".mp4", ".json"))
            keypoint_dir = os.path.join(subdir_path, "KEYPOINT_DIR", video_file.replace(".mp4", ""))

            video_2_json(video_path, json_output_path, description, keypoint_dir, device)

if __name__ == "__main__":
    main()


⚙️ 使用裝置：cuda


 13%|█▎        | 27/203 [00:00<00:00, 266.94it/s]

data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0001 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0002 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0003 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0004 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0005 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0006 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0007 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0008 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0009 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0010 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0011 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0012 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0013 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0014 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0015 已存在，跳過處理。
data/Gerrit_Cole_CH_videos_4S/KEYPOINT_D

  0%|          | 0/238 [00:00<?, ?it/s]

👤 Frame 0: 設定目標人物 ID = 1


...

👤 Frame 0: 設定目標人物 ID = 1
✅ Detectron2 處理完畢，骨架資料已儲存。



 28%|██▊       | 57/203 [44:25<6:09:27, 151.83s/it]

0 : 24.933853149414062 717.4249267578125 710.7879028320312
出手條件不滿足
1 : 25.818771362304688 717.5052490234375 711.7584228515625
2 : 24.957077026367188 717.5045776367188 711.759033203125
出手條件不滿足
3 : 26.660873413085938 718.0983276367188 711.4503173828125
4 : 26.724029541015625 717.5658569335938 710.8655395507812
5 : 26.785491943359375 716.8642578125 711.1165771484375
6 : 26.766754150390625 717.5826416015625 710.8875122070312
7 : 25.771148681640625 718.10693359375 711.4514770507812
8 : 25.88201904296875 717.468994140625 710.7755737304688
9 : 27.565444946289062 717.9174194335938 711.2249755859375
10 : 27.805191040039062 718.1417846679688 711.4461059570312
11 : 27.691665649414062 718.4481811523438 711.7316284179688
12 : 26.643325805664062 718.51708984375 711.7998046875
13 : 26.789779663085938 718.587890625 711.8696899414062
14 : 26.826904296875 718.515625 711.8088989257812
15 : 26.598007202148438 718.7360229492188 712.0172729492188
16 : 26.745880126953125 718.4070434570312 711.711181640625
17

  0%|          | 0/239 [00:00<?, ?it/s]

👤 Frame 0: 設定目標人物 ID = 1
